# 04 - Advanced Forecasting Models

Implement advanced deep learning models not covered in class:
1. N-BEATS (Neural Basis Expansion Analysis for Time Series)
2. XGBoost (Tree-based approach)
3. Model comparison and analysis

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import yaml

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 6)

from src.evaluation.metrics import regression_metrics
from src.utils.seed import set_seed

with open('../config/project.yaml') as f:
    config = yaml.safe_load(f)

set_seed(config['random_seed'])
print('Setup complete')

## 1. Load Data

In [ ]:
data_dir = Path('../data/processed')
train_df = pd.read_parquet(data_dir / 'train.parquet')
val_df = pd.read_parquet(data_dir / 'val.parquet')
test_df = pd.read_parquet(data_dir / 'test.parquet')

target = config['project']['target_variable']
print(f'Data loaded. Target: {target}')
print(f'Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}')

## 2. XGBoost Model

Gradient boosted trees with lag and rolling features.

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

# Prepare features
feature_cols = [col for col in train_df.columns if col not in [target, 'is_outlier']]
print(f'Using {len(feature_cols)} features')

# Handle NaN values
X_train = train_df[feature_cols].fillna(method='ffill').fillna(method='bfill')
y_train = train_df[target]

X_val = val_df[feature_cols].fillna(method='ffill').fillna(method='bfill')
y_val = val_df[target]

X_test = test_df[feature_cols].fillna(method='ffill').fillna(method='bfill')
y_test = test_df[target]

print(f'Training XGBoost...')
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=5,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=config['random_seed'],
    n_jobs=-1
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=50,
    verbose=False
)

print('XGBoost training complete')

In [ ]:
# Predictions
xgb_pred_val = xgb_model.predict(X_val)
xgb_pred_test = xgb_model.predict(X_test)

# Metrics
xgb_metrics_val = regression_metrics(y_val, xgb_pred_val)
xgb_metrics_test = regression_metrics(y_test, xgb_pred_test)

print('XGBoost - Validation:')
print(xgb_metrics_val)
print('\nXGBoost - Test:')
print(xgb_metrics_test)

results = {'xgboost': {'val': xgb_metrics_val, 'test': xgb_metrics_test}}

In [ ]:
# Feature importance
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False).head(20)

fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(importance['feature'], importance['importance'], alpha=0.7, edgecolor='black')
ax.set_xlabel('Importance')
ax.set_title('XGBoost Top 20 Feature Importances', fontsize=12, fontweight='bold')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('../reports/xgboost_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 1, figsize=(18, 10))

axes[0].plot(val_df.index, y_val, label='Actual', linewidth=2)
axes[0].plot(val_df.index, xgb_pred_val, label='XGBoost Forecast', linewidth=2, linestyle='--')
axes[0].set_title('XGBoost Forecast - Validation Set', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Power (kW)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(test_df.index, y_test, label='Actual', linewidth=2)
axes[1].plot(test_df.index, xgb_pred_test, label='XGBoost Forecast', linewidth=2, linestyle='--')
axes[1].set_title('XGBoost Forecast - Test Set', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Power (kW)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/xgboost_forecast.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. LightGBM Model

Another gradient boosting variant for comparison.

In [ ]:
import lightgbm as lgb

print('Training LightGBM...')
lgb_model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=5,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=config['random_seed'],
    n_jobs=-1,
    verbose=-1
)

lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
)

print('LightGBM training complete')

In [ ]:
# Predictions
lgb_pred_val = lgb_model.predict(X_val)
lgb_pred_test = lgb_model.predict(X_test)

# Metrics
lgb_metrics_val = regression_metrics(y_val, lgb_pred_val)
lgb_metrics_test = regression_metrics(y_test, lgb_pred_test)

print('LightGBM - Validation:')
print(lgb_metrics_val)
print('\nLightGBM - Test:')
print(lgb_metrics_test)

results['lightgbm'] = {'val': lgb_metrics_val, 'test': lgb_metrics_test}

## 4. Random Forest

Ensemble of decision trees.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

print('Training Random Forest...')
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=config['random_seed'],
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
print('Random Forest training complete')

# Predictions
rf_pred_val = rf_model.predict(X_val)
rf_pred_test = rf_model.predict(X_test)

# Metrics
rf_metrics_val = regression_metrics(y_val, rf_pred_val)
rf_metrics_test = regression_metrics(y_test, rf_pred_test)

print('Random Forest - Validation:')
print(rf_metrics_val)
print('\nRandom Forest - Test:')
print(rf_metrics_test)

results['random_forest'] = {'val': rf_metrics_val, 'test': rf_metrics_test}

## 5. Model Comparison

In [ ]:
# Create comparison table
comparison_data = []
for model_name, metrics in results.items():
    comparison_data.append({
        'Model': model_name.upper(),
        'Val MAE': metrics['val']['mae'],
        'Val RMSE': metrics['val']['rmse'],
        'Val MAPE': metrics['val']['mape'],
        'Val R²': metrics['val']['r2'],
        'Test MAE': metrics['test']['mae'],
        'Test RMSE': metrics['test']['rmse'],
        'Test MAPE': metrics['test']['mape'],
        'Test R²': metrics['test']['r2']
    })

comparison_df = pd.DataFrame(comparison_data).round(4)
print('\nAdvanced Model Comparison:')
print(comparison_df.to_string(index=False))

comparison_df.to_csv('../reports/advanced_models_comparison.csv', index=False)
print('\nResults saved')

## Summary

### Advanced Models Implemented:
1. ✅ XGBoost (Gradient Boosted Trees)
2. ✅ LightGBM (Efficient Gradient Boosting)
3. ✅ Random Forest (Ensemble)

### Justification:
**XGBoost and LightGBM** were chosen as advanced models because:
- Excel at capturing non-linear relationships in time series
- Handle missing values naturally
- Provide feature importance for interpretability
- Proven track record in forecasting competitions
- Efficient computation compared to deep learning

### Key Advantages:
- **Feature Importance**: Unlike black-box models, tree-based methods show which features drive predictions
- **Robustness**: Less sensitive to outliers and missing values
- **Speed**: Faster training than deep neural networks
- **Interpretability**: Decision paths can be traced

### Expected Performance:
- Should outperform classical statistical methods (SARIMA)
- Competitive with or better than LSTM for tabular time series
- Excellent for capturing complex lag interactions

Proceed to notebook 05_model_comparison.ipynb for comprehensive comparison.